In [1]:
import pandas as pd
import numpy as np
import sqlite3

In [2]:
conn = sqlite3.connect('car_rates.db')
sql = 'SELECT dt, engine, car_body, costs, car_details, equipment, driving_features, car, grade  FROM car_rates'
data = pd.read_sql(sql, conn)

conn.close()

print(data)

                      dt  engine  car_body  costs  car_details  equipment  \
0    2020-05-28 15:54:01    31.0      57.0   20.0         37.0       75.0   
1    2020-05-28 15:54:01    31.0      57.0   20.0         37.0       75.0   
2    2020-05-28 15:54:01    31.0      57.0   20.0         37.0       75.0   
3    2020-05-28 15:54:01    31.0      57.0   20.0         37.0       75.0   
4    2020-05-28 15:54:01    31.0      57.0   20.0         37.0       75.0   
..                   ...     ...       ...    ...          ...        ...   
600  2020-05-28 15:57:23    29.0      51.0   54.0         47.0       41.0   
601  2020-05-28 15:57:24    37.0      31.0   14.0         52.0       32.0   
602  2020-05-28 15:57:24    37.0      31.0   14.0         52.0       32.0   
603  2020-05-28 15:57:24    37.0      31.0   14.0         52.0       32.0   
604  2020-05-28 15:57:24    37.0      31.0   14.0         52.0       32.0   

     driving_features             car  grade  
0                -1.0      O

In [3]:
data.grade.describe()

count    605.000000
mean       3.024793
std        1.822699
min        1.000000
25%        1.000000
50%        3.000000
75%        5.000000
max        5.000000
Name: grade, dtype: float64

In [4]:
data.grade.hist(bins=10)

In [5]:
data.groupby('car').count()['grade'].describe()

count     11.000000
mean      55.000000
std       51.774511
min        3.000000
25%       10.500000
50%       37.000000
75%       84.500000
max      148.000000
Name: grade, dtype: float64

In [6]:
data['dt']

0      2020-05-28 15:54:01
1      2020-05-28 15:54:01
2      2020-05-28 15:54:01
3      2020-05-28 15:54:01
4      2020-05-28 15:54:01
              ...         
600    2020-05-28 15:57:23
601    2020-05-28 15:57:24
602    2020-05-28 15:57:24
603    2020-05-28 15:57:24
604    2020-05-28 15:57:24
Name: dt, Length: 605, dtype: object

In [7]:
from sklearn.metrics.pairwise import cosine_similarity

def prepare_user_column(dt):
    dt_to_user = {}

    c = 1
    for d in dt.drop_duplicates():
        dt_to_user[d] = c
        c += 1

    user_column = []

    for e in dt:
        user_column.append(dt_to_user[e])

    return user_column

def get_similar_cars(car, similarity_df, n_neighbors=8):
    similar_ids = similarity_df.loc[car].sort_values(ascending=False)[1:n_neighbors+1].reset_index()
    return similar_ids

def prepare_matrix(data, column_name):
    return data.pivot_table(index='car', columns='userId', values=column_name, fill_value=0)

def get_cosine_similarity(matrix):
    cosine_similarity_cf_mx = cosine_similarity(matrix)
    return pd.DataFrame(cosine_similarity_cf_mx, columns=matrix.index, index=matrix.index)

def process(data, car_name):
    user_column = prepare_user_column(data['dt'])
    data_s = data.assign(userId=pd.Series(user_column).values)
    column_names = ['engine', 'car_body', 'costs', 'car_details', 'equipment', 'driving_features', 'grade']
    results = {}
    
    for c in column_names:
        mx = prepare_matrix(data_s, c)
        sim_mx = get_cosine_similarity(mx)
        sim_cars = get_similar_cars(car_name, sim_mx)
        results[c] = sim_cars

    return results

In [8]:
user_column = prepare_user_column(data['dt'])
data = data.assign(userId=pd.Series(user_column).values)
sample_car = data['car'].sample().to_list()[0]
sample_car

'Opel Grossland'

In [18]:
def get_best_match(data, car):
    processing_result = process(data, car)

    best_cars = {}
    for _, v in processing_result.items():
        cars = v['car'].to_list()
        values = v[car].to_list()
        for index, car_name in enumerate(cars):
            tmp = 0
            if car_name in best_cars:
                tmp = best_cars[car_name]
            tmp += values[index]
            best_cars[car_name] = tmp
    
    result = sorted(best_cars.items(), key=lambda item: item[1], reverse=True)[:5]
    return result

def get_recommendation(d, predicted_cars):
    result = {}
    car_names = predicted_cars.keys()
    for c in car_names:

        if c not in d['car'].drop_duplicates().tolist():
            continue

        distinct_cars = [a for (a, b) in get_best_match(d, c)]
        
        filtered_cars = []
        for e in distinct_cars:
            if e not in car_names:
                filtered_cars.append(e)
                
        result[c] = filtered_cars

    return result

pc = {'Fiat 500': 24, 'VW Touareg': 24, 'Skoda Scala': 3, 'VW T-ROC': 1, 'Opel Grossland': 3}

get_recommendation(data, pc)

{'Fiat 500': ['Opel Astra'],
 'VW Touareg': ['Opel Astra'],
 'Skoda Scala': ['Opel Astra'],
 'VW T-ROC': ['Opel Astra', 'VW Polo'],
 'Opel Grossland': ['Opel Corsa', 'Fiat 500X', 'Opel Astra']}

In [23]:
user_column = prepare_user_column(data['dt'])
data_s = data.assign(userId=pd.Series(user_column).values)
column_names = ['engine', 'car_body', 'costs', 'car_details', 'equipment', 'driving_features', 'grade']
results = {}
    
mx = prepare_matrix(data_s, 'grade')
sim_mx = get_cosine_similarity(mx)
sim_cars = get_similar_cars('Fiat 500', sim_mx)
mx
sim_cars

,car,Fiat 500
0,VW Touareg,0.878969
1,Opel Astra,0.572906
2,Opel Grossland,0.425364
3,Skoda Scala,0.405591
4,VW T-ROC,0.190370
5,Opel Corsa,0.151136
6,VW Polo,0.061872
7,Fiat Tipo,0.055334


In [ ]:
def get_all_similar_predictions(car_elements_list):
    historical_data = get_historical_data()
    result = {}
    for historical_row in historical_data:
        strong_similarity = 0
        small_similarity = 0
        for index, element in enumerate(car_elements_list):
            if are_strong_similar_values(element, historical_row[index]):
                strong_similarity += 1
            if are_small_similar_values(element, historical_row[index]):
                small_similarity += 1
        if strong_similarity > 3:
            result.update({historical_row[-2], historical_row[-1]})
        elif small_similarity > 5:
            result.update({historical_row[-2], historical_row[-1]})
    return result